In [4]:
!python yolov5_ws/yolov5/train.py --img 640 --batch 20 --epochs 1 --device "cuda:0" --data yolov5_ws/yolov5/dataset.yaml --weights yolov5_ws/yolov5/yolov5s.pt

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


train: weights=yolov5_ws/yolov5/yolov5s.pt, cfg=, data=yolov5_ws/yolov5/dataset.yaml, hyp=yolov5_ws\yolov5\data\hyps\hyp.scratch-low.yaml, epochs=1, batch_size=20, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=cuda:0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5_ws\yolov5\runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  2022-12-5 Python-3.9.15 torch-1.13.0 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, h

In [ ]:
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import cv2

# Load the images and labels
images = []
labels = []

with open("dataset.txt", "r") as file:
    for line in file:
        line = line.strip().split()
        images.append(cv2.imread(line[0]))
        labels.append(line[1:])

# Convert the images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Define the augmentation sequence
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of the images
    iaa.Affine(rotate=(-45, 45)),  # rotate the images by a random degree between -45 and 45
    iaa.AdditiveGaussianNoise(scale=(0, 0.1*255)),  # add random noise to the images
    iaa.Multiply((0.8, 1.2)),  # adjust the brightness of the images
])

# Augment the images and labels
images_aug, labels_aug = seq(images=images, annotations=labels)

# Save the augmented images and labels
for i, (image, label) in enumerate(zip(images_aug, labels_aug)):
    cv2.imwrite("augmented_images/image_{}.jpg".format(i), image)
    with open("augmented_labels/label_{}.txt".format(i), "w") as file:
        file.write(" ".join(label))

In [ ]:
import pandas as pd
from sklearn.utils import resample

# Load the dataset
df = pd.read_csv("dataset.csv")

# Separate the majority and minority classes
df_majority = df[df.target.isin([0, 1])]
df_minority = df[df.target.isin([2, 3, 4])]

# Oversample the minority class
df_minority_oversampled = resample(df_minority,
                                    replace=True,
                                    n_samples=len(df_majority),
                                    random_state=123)

# Undersample the majority class
df_majority_undersampled = resample(df_majority,
                                     replace=False,
                                     n_samples=len(df_minority),
                                     random_state=123)

# Combine the oversampled minority class with the undersampled majority class
df_balanced = pd.concat([df_majority_undersampled, df_minority_oversampled])

# Shuffle the rows
df_balanced = df_balanced.sample(frac=1, random_state=123).reset_index(drop=True)

# Print the balanced dataset
print(df_balanced)
